In [2]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Input
from tensorflow.keras.layers import Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.initializers import glorot_uniform
import os
import pandas as pd
from dataset_utilits import load_imagesPath_ages_sex
from sklearn.model_selection import train_test_split
import random
import cv2
from sklearn.decomposition import PCA
from operator import itemgetter
from inception_utilits import conv2d_bn, stem_block, inception_a_block, inception_b_block, inception_c_block, reduction_a_block, reduction_b_block
from shapely.geometry import Polygon
import random


In [4]:
train_truth = pd.read_csv('./data.nosync/train/train.csv')
validation_truth = pd.read_csv('./data.nosync/validation/Validation Dataset.csv')
test_truth = pd.read_excel('./data.nosync/test/Bone age ground truth.xlsx')
IMG_SHAPE = (299, 299, 1)

In [6]:
path_data_dir = 'data.nosync'
full_path = os.path.join(os.getcwd(), path_data_dir)
X_train, age_train, sex_train, X_validation, age_validation, sex_validation, X_test, age_test, sex_test = load_imagesPath_ages_sex(full_path, train_truth, validation_truth, test_truth, 500)

age_train = [x / 200. for x in age_train]
age_validation = [x / 200. for x in age_validation]
age_train_df = pd.DataFrame(age_train, columns=['age'])
age_validation_df = pd.DataFrame(age_validation, columns=['age'])
age_train_df = age_train_df.astype('float32')
age_validation_df = age_validation_df.astype('float32')

prendo le image_names da  /Users/pietroferrazzi/Desktop/uni/human_data/project/HDA-bone-age-prediction/data.nosync/train/images
prendo le image_names da  /Users/pietroferrazzi/Desktop/uni/human_data/project/HDA-bone-age-prediction/data.nosync/validation/images
prendo le image_names da  /Users/pietroferrazzi/Desktop/uni/human_data/project/HDA-bone-age-prediction/data.nosync/test/images


In [7]:
print("X shape:", len(X_test))
print("X train shape:", len(X_train))
print("X val shape:", len(X_validation))

X shape: 200
X train shape: 500
X val shape: 500


In [7]:
datagen_train = ImageDataGenerator(preprocessing_function = lambda x: x/255.,
                             width_shift_range=4,  # randomly shift images horizontally
                             height_shift_range=4,  # randomly shift images vertically 
                             horizontal_flip=True,  # randomly flip images
                             vertical_flip=True)  # randomly flip images
datagen_validation = ImageDataGenerator(preprocessing_function=lambda x: x/255.)
datagen_test = ImageDataGenerator(preprocessing_function = lambda x: x/255.)

In [7]:
batch_size = 32
train_truth = pd.read_csv('./data.nosync/train/train.csv')
validation_truth = pd.read_csv('./data.nosync/validation/Validation Dataset.csv')
train_truth['boneage'] = train_truth['boneage']/200
train_truth['id'] = train_truth['id'].astype(str) + '.png'
validation_truth['Bone Age (months)'] = validation_truth['Bone Age (months)']/200
train_generator = datagen_train.flow_from_dataframe(dataframe=train_truth, directory='./data.nosync/train/images', 
                                                    x_col="id",
                                                    y_col="boneage",  
                                                    class_mode="other", target_size=(IMG_SHAPE[0], IMG_SHAPE[1]), 
                                                    batch_size=batch_size)
validation_truth['Image ID'] = validation_truth['Image ID'].astype(str) + '.png'
validation_generator = datagen_validation.flow_from_dataframe(dataframe=validation_truth, directory='./data.nosync/validation/images', 
                                                              x_col="Image ID", 
                                                              y_col="Bone Age (months)", 
                                                              class_mode="other", target_size=(IMG_SHAPE[0], IMG_SHAPE[1]), 
                                                              batch_size=batch_size)
batch_size = 32
train_steps = int(np.ceil( train_generator.n / batch_size))
validation_steps = int(np.ceil(validation_generator.n / batch_size))
# plt.imshow(train_generator.next()[0][0])

Found 12611 validated image filenames.
Found 1425 validated image filenames.


### Second branch data

In [8]:
dist_btw_bones1 = np.random.uniform(1, 100, size=len(sex_train))
dist_btw_bones2 = np.random.uniform(1, 100, size=len(sex_train))
dist_btw_bones3 = np.random.uniform(1, 100, size=len(sex_train))
dist_btw_bones4 = np.random.uniform(1, 100, size=len(sex_train))

dist_btw_bones1 = np.random.normal(age_train, 1, size=len(sex_train))
dist_btw_bones2 = np.random.normal(age_train, 1, size=len(sex_train))
dist_btw_bones3 = np.random.normal(age_train, 1, size=len(sex_train))
dist_btw_bones4 = np.random.normal(age_train, 1, size=len(sex_train))

second_branch_df_train = pd.DataFrame(list(zip(sex_train, dist_btw_bones1, dist_btw_bones2, dist_btw_bones3, dist_btw_bones4)),
                            columns =['sex', 'd1', 'd2', 'd3', 'd4'])
second_branch_df_train.sex = second_branch_df_train.sex.replace({True: 1., False: 0.})

dist_btw_bones1_val = np.random.normal(age_validation, 10, size=len(sex_validation))
dist_btw_bones2_val = np.random.normal(age_validation, 10, size=len(sex_validation))
dist_btw_bones3_val = np.random.normal(age_validation, 10, size=len(sex_validation))
dist_btw_bones4_val = np.random.normal(age_validation, 10, size=len(sex_validation))
second_branch_df_val = pd.DataFrame(list(zip(sex_validation, dist_btw_bones1_val, dist_btw_bones2_val, dist_btw_bones3_val, dist_btw_bones4_val)),
                            columns =['sex', 'd1', 'd2', 'd3', 'd4'])
second_branch_df_val.sex = second_branch_df_val.sex.replace({True: 1, False: 0})


Try if the second branch works by itself:

In [9]:
# def create_dataset_branch2(second_branch_df, ages, batch_size, shuffle, cache_file=None):

#     # Create a Dataset object
#     dataset = tf.data.Dataset.from_tensor_slices((second_branch_df, ages))

#     # Cache dataset
#     if cache_file:
#         dataset = dataset.cache(cache_file)


#     # Shuffle
#     if shuffle:
#         dataset = dataset.shuffle(len(second_branch_df))

#     # Repeat the dataset indefinitely
#     dataset = dataset.repeat()

#     # Batch
#     dataset = dataset.batch(batch_size=batch_size)

#     # Prefetch
#     dataset = dataset.prefetch(buffer_size=1)

#     return dataset

# batch_size = 32
# train_dataset_branch_2 = create_dataset_branch2(second_branch_df = second_branch_df_train, 
#                     ages = age_train, 
#                     batch_size = batch_size, 
#                     shuffle = False )  

# validation_dataset_branch_2 = create_dataset_branch2(second_branch_df = second_branch_df_val, 
#                     ages = age_validation, 
#                     batch_size = batch_size, 
#                     shuffle = False ) 

# def dense_branch_2(X_input):
#     X = Dense(64, activation='relu', name='first_dense_branch_2')(X_input)
#     X = Dense(32, activation='relu', name='second_dense_branch_2')(X)
#     X = Flatten()(X)
#     # max_pooling o global_pooling (valore unico) [regolarizza vs overfitting] o dense più piccolo - > dense + pooling
#     return X


# def model_assembly_example(input_shape_dataset):
#     # branch 2
#     X_input_branch2 = Input(input_shape_dataset)
#     branch2 = dense_branch_2(X_input_branch2)
#     X = Dense(1, activation = 'relu', name='final')(branch2)


#     # Create model
#     model = Model(inputs = X_input_branch2, outputs = X, name='branch2_attempt')
#     return model

# train_steps = int(np.ceil(len(sex_train) / batch_size))
# val_steps = int(np.ceil(len(sex_validation) / batch_size))

# model = model_assembly_example(input_shape_dataset=5)

# model.compile(optimizer="adam", loss="mean_squared_error", metrics=tf.keras.metrics.mean_squared_error)

# # Create a callback for early stopping 
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# history = model.fit(train_dataset_branch_2, 
#                     validation_data = validation_dataset_branch_2, 
#                     epochs=100, 
#                     steps_per_epoch=train_steps,
#                     validation_steps=val_steps,
#                     callbacks=[callback])

## MODEL

In [10]:
def Inceptionv4(X_input): #  (input_shape):
    """
    Implementation of the Inception-v4 architecture

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # # Define the input as a tensor with shape input_shape (1 line)
    # X_input = Input(input_shape)

    # Call the above functions for the stem, inception-a, reduction-a, inception-b, reduction-b and inception-c blocks
    X = stem_block(X_input)

    # Four Inception A blocks
    X = inception_a_block(X, 'a1')
    X = inception_a_block(X, 'a2')
    X = inception_a_block(X, 'a3')
    X = inception_a_block(X, 'a4')

    # Reduction A block
    X = reduction_a_block(X)

    # Seven Inception B blocks
    X = inception_b_block(X, 'b1')
    X = inception_b_block(X, 'b2')
    X = inception_b_block(X, 'b3')
    X = inception_b_block(X, 'b4')
    X = inception_b_block(X, 'b5')
    X = inception_b_block(X, 'b6')
    X = inception_b_block(X, 'b7')

    # Reduction B block
    X = reduction_b_block(X)

    # Three Inception C blocks
    X = inception_c_block(X, 'c1')
    X = inception_c_block(X, 'c2')
    X = inception_c_block(X, 'c3')

    # AVGPOOL (1 line). Use "X = AveragePooling2D(...)(X)"
    kernel_pooling = X.get_shape()[1:3]
    X = AveragePooling2D(kernel_pooling, name='avg_pool')(X)
    X = Flatten()(X)

    # Dropout
    X = Dropout(rate = 0.2)(X)

    # Output layer
    # X = Dense(1, activation='relu', name='fc')(X)
    
    
    # # Create model
    # model = Model(inputs = X_input, outputs = X, name='Inceptionv4')

    # return model
    return X

In [11]:
def dense_branch_2(X_input):
    X = Dense(64, activation='relu', name='first_dense_branch_2')(X_input)
    X = Dense(32, activation='relu', name='second_dense_branch_2')(X)
    X = Flatten()(X)
    # max_pooling o global_pooling (valore unico) [regolarizza vs overfitting] o dense più piccolo - > dense + pooling
    return X


In [11]:
# # IF YOU WANT TO LOAD THE DATA BEFORE HAND 
# full_df_train = second_branch_df_train
# full_df_train['img_path'] = X_train

# full_df_val = second_branch_df_val
# full_df_val['img_path'] = X_validation

# full_df_train['img'] = full_df_train.apply(lambda row: load_image(row['img_path'], IMG_SHAPE), axis = 1) 
# full_df_train = full_df_train.drop(['img_path'], axis=1)

# full_df_val['img'] = full_df_val.apply(lambda row: load_image(row['img_path'], IMG_SHAPE), axis = 1) 
# full_df_val = full_df_val.drop(['img_path'], axis=1)

In [12]:
# def process_image(path_df, age):

#     num_row = IMG_SHAPE[1]
#     num_col = IMG_SHAPE[0]

#     img_path, df = path_df
#     img_path = str(img_path)
#     if isinstance(img_path, bytes):
#         img_path = img_path.decode()
    
#     img = cv2.imread(img_path, cv2.IMREAD_COLOR)
#     img = np.array(cv2.resize(img, (num_row, num_col)), dtype='float32')

#     # Normalize image
#     img = img/255.0
#     #return img, df
#     return (img, df), age

def process_image_2(path_df, label):
    # Desired size
    # num_row = IMG_SHAPE[1]
    # num_col = IMG_SHAPE[0]
    size = IMG_SHAPE[1]
    path, df = path_df

    # Get the image
    img = tf.io.read_file(path)
    # Decode the PNG
    img = tf.image.decode_png(img)
    # Resize image
    img = tf.image.resize(img, (size, size))
    # Reshape image (this is not necessary but I do it so that I don't need to be modifying the shape in the input layer)
    #img = tf.reshape(img, [size, size, 3])
    # Cast image to float32
    img = tf.cast(img, tf.float32)
    # Normalize image
    img = img/255.0

    return (img, df), label

In [20]:
def create_dataset_images(img_paths, second_branch_df, ages, batch_size, shuffle, cache_file=None):

    # Create a Dataset object
    second_branch_df = second_branch_df.astype('float32')
    dataset = tf.data.Dataset.from_tensor_slices(((img_paths, second_branch_df), ages)).map(process_image_2)

    # Cache dataset
    if cache_file:
        dataset = dataset.cache(cache_file)

    # Shuffle
    if shuffle:
        dataset = dataset.shuffle(len(img_paths))

    # Repeat the dataset indefinitely
    dataset = dataset.repeat()

    # Batch
    dataset = dataset.batch(batch_size=batch_size)

    # Prefetch
    dataset = dataset.prefetch(buffer_size=1)

    return dataset

batch_size = 32
train_dataset = create_dataset_images(  img_paths = X_train, 
                                        second_branch_df = second_branch_df_train,
                                        ages = age_train_df, 
                                        batch_size = batch_size, 
                                        shuffle = False,
                                        cache_file='cache_train' )  

validation_dataset = create_dataset_images(img_paths = X_validation, 
                                            second_branch_df = second_branch_df_val,
                                            ages = age_validation_df, 
                                            batch_size = batch_size, 
                                            shuffle = False ,
                                        cache_file='cache_val') 

train_steps = int(np.ceil(len(X_train) / batch_size))
validation_steps = int(np.ceil(len(X_validation) / batch_size))

In [21]:
def model_assembly(input_shape_img, input_shape_dataset):

    # Branch 1
    X_input_branch1 = Input(input_shape_img)
    print(input_shape_img)
    branch1 = Inceptionv4(X_input_branch1)

    # # Branch 2
    X_input_branch2 = Input(input_shape_dataset)
    branch2 = dense_branch_2(X_input_branch2)
    
    # # Concatenate branch1 and branch2
    X = tf.concat(values=[branch1, branch2], axis=1)
    X = Dense(1000, activation='relu', name = 'final_dense_1')(X)
    X = Dense(1000, activation='relu', name = 'final_dense_2')(X)
    X_out = Dense(1, activation = 'linear', name='final')(X)

    # Create model
    model = Model(inputs = (X_input_branch1, X_input_branch2) , outputs = X_out, name='model0') #  X_input_branch2
    return model

In [22]:
model = model_assembly(input_shape_img = IMG_SHAPE, input_shape_dataset = 5)

model.compile(optimizer="adam", loss="mean_squared_error", metrics=tf.keras.metrics.mean_squared_error)

# Create a callback for early stopping 
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

(299, 299, 1)


In [23]:
# Fit the model on batches with real-time data augmentation:
history = model.fit(train_dataset,
                    validation_data = validation_dataset, 
                    epochs=10, 
                    steps_per_epoch=train_steps,
                    validation_steps=validation_steps,
                    callbacks=[callback])

Epoch 1/10
14/16 [=========================>....] - ETA: 19s - loss: 7.3133 - mean_squared_error: 7.3133

2023-02-10 11:31:46.838983: W tensorflow/core/kernels/data/cache_dataset_ops.cc:296] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


16/16 [==============================] - ETA: 0s - loss: 6.4172 - mean_squared_error: 6.4172 

2023-02-10 11:32:54.456369: W tensorflow/core/kernels/data/cache_dataset_ops.cc:296] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


16/16 [==============================] - 217s 13s/step - loss: 6.4172 - mean_squared_error: 6.4172 - val_loss: 0.9809 - val_mean_squared_error: 0.9809
Epoch 2/10
16/16 [==============================] - 195s 12s/step - loss: 0.0686 - mean_squared_error: 0.0686 - val_loss: 0.4297 - val_mean_squared_error: 0.4297
Epoch 3/10
16/16 [==============================] - 190s 12s/step - loss: 0.0533 - mean_squared_error: 0.0533 - val_loss: 0.2744 - val_mean_squared_error: 0.2744
Epoch 4/10
16/16 [==============================] - 191s 12s/step - loss: 0.0499 - mean_squared_error: 0.0499 - val_loss: 0.1460 - val_mean_squared_error: 0.1460
Epoch 5/10
16/16 [==============================] - 195s 12s/step - loss: 0.0432 - mean_squared_error: 0.0432 - val_loss: 0.1222 - val_mean_squared_error: 0.1222
Epoch 6/10
 9/16 [===============>..............] - ETA: 1:05 - loss: 0.0406 - mean_squared_error: 0.0406

KeyboardInterrupt: 